In [246]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score, classification_report, log_loss, accuracy_score
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier

In [247]:
df = pd.read_parquet('data.parquet')
print(df.shape)
df.head()

(8012, 92)


,is_title_fight,winner,no_of_rounds,weight_class_bantamweight,weight_class_catch_weight,weight_class_featherweight,weight_class_flyweight,weight_class_heavyweight,weight_class_light_heavyweight,weight_class_lightweight,...,b_wins_by_ko_tko,b_wins_by_sub,b_wins_by_m_dec,b_wins_by_s_dec,b_wins_by_u_dec,b_age,b_is_debut,delta_age,delta_height,delta_reach
0,False,Red,1,False,False,False,False,False,False,False,...,0,0,0,0,0,25,1,5,2.0,3.0
1,False,Red,1,False,False,False,False,False,False,False,...,0,0,0,0,0,32,1,-2,6.0,0.0
2,False,Red,2,False,False,False,False,True,False,False,...,0,0,0,0,0,23,1,3,-2.0,0.0
3,True,Red,5,False,False,False,False,False,False,True,...,0,0,0,0,0,25,1,1,0.0,0.0
4,False,Red,2,False,False,False,False,False,False,False,...,0,0,0,0,0,27,1,-3,-4.0,-3.0


In [248]:
le = LabelEncoder()

TARGET = 'winner'

df[TARGET] = le.fit_transform(df[TARGET])

In [249]:
y = df[TARGET]
X = df.drop(columns='winner')

In [250]:
n = len(df)
print(len(df))
split_idx = int(n * 0.8095)
train_df = df.iloc[:split_idx]
test_df = df.iloc[split_idx:]

8012


In [251]:
X_train = X.iloc[:split_idx]
X_test = X.iloc[split_idx:]

y_train = y.iloc[:split_idx]
y_test = y.iloc[split_idx:]

In [252]:
print(f'X_train : {X_train.shape}')
print(f'y_train : {y_train.shape}')
print(f'X_test : {X_test.shape}')
print(f'y_test : {y_test.shape}')

X_train : (6485, 91)
y_train : (6485,)
X_test : (1527, 91)
y_test : (1527,)


In [253]:
assert train_df.index.max() < test_df.index.min()
assert TARGET not in X_train.columns

print(y_train.unique())
print(y_train.mean(), y_test.mean())

[1 0]
0.6485736314572089 0.5579567779960707


In [ ]:
rf = RandomForestClassifier(
  n_estimators=300, 
  oob_score=True, 
  random_state=43)

rf.fit(X_train, y_train)

[(50, 0.6220222569987829), (100, 0.6337802121370196), (200, 0.6362389149713094), (300, 0.6437297861241524), (500, 0.6455338202051817), (800, 0.6457163971483221)]


In [255]:
rf.oob_score_

0.6649190439475713

In [256]:
y_preds = rf.predict(X_test)

In [257]:
accuracy_score(y_test, y_preds)

0.6064178127046497

In [258]:
roc_auc_score(y_test, rf.predict_proba(X_test)[:, 1])

0.6457163971483221